In [1]:
import pandas as pd
import numpy as np
import PyUber

In [2]:
sql = r"""SELECT 
          a0.lot7 AS lot7
         ,a0.devrevstep AS devrevstep
         ,(SELECT lrc99.last_pass FROM F_LOT_RUN_CARD lrc99 where lrc99.lot =a0.lot AND lrc99.operation = a0.operation AND lrc99.site_prevout_date=a0.prev_moveout_time and rownum<=1) AS last_pass
         ,a0.rework_latest_flag AS rework_latest_flag
         ,a0.lot_rework_flag AS rework_flag
         ,a1.entity AS entity
         ,a1.ceid AS ceid
         ,a2.monitor_set_name AS monitor_set_name
         ,To_Char(a2.data_collection_time,'yyyy-mm-dd hh24:mi:ss') AS data_collect_date
         ,a2.area AS area
         ,a2.monitor_process AS monitor_process
         ,a2.status AS mon_set_status
         ,a2.spc_data_id AS spc_data_id
         ,a2.violation_flag AS violation_flag
         ,a2.latest_flag AS mon_set_latest_flag
         ,a5.chart_point_seq AS chart_point_seq
         ,a5.value AS chart_value
         ,a5.process_chamber AS process_chamber
         ,a5.wafer AS chart_wafer
         ,a5.latest_flag AS chart_pt_latest_flag
         ,a5.status AS chart_pt_status
         ,a5.chart_type AS chart_type
         ,a5.spc_chart_category AS spc_chart_category
         ,a5.spc_chart_subset AS spc_chart_subset
         ,a10.lo_control_lmt AS lo_control_lmt
         ,a10.target AS target
         ,a10.up_control_lmt AS up_control_lmt
         ,a2.test_name AS test_name
         ,a0.operation AS spc_operation
         ,a5.wafer3 AS chart_wafer3
         ,a2.monitor_type AS monitor_type
         ,a3.parameter_class AS parameter_class
         ,a9.chart_parameter AS chart_parameter
         ,a9.chart_on AS chart_on
         ,a3.measurement_set_name AS measurement_set_name
         ,a11.
FROM 
P_SPC_LOT a0
LEFT JOIN P_SPC_ENTITY a1 ON a1.spcs_id = a0.spcs_id AND a1.entity_sequence=1
INNER JOIN P_SPC_SESSION a2 ON a2.spcs_id = a0.spcs_id AND a2.data_collection_time = a0.data_collection_time
INNER JOIN P_SPC_MEASUREMENT_SET a3 ON a3.spcs_id = a2.spcs_id
LEFT JOIN P_SPC_CHART_POINT a5 ON a5.spcs_id = a3.spcs_id AND a5.measurement_set_name = a3.measurement_set_name
LEFT JOIN P_SPC_CHART a9 ON a9.chart_id = a5.chart_id
LEFT JOIN P_SPC_CHART_LIMIT a10 ON a10.chart_id = a5.chart_id AND a10.limit_id = a5.limit_id
LEFT JOIN P_SPC_CHARTPOINT_MEASUREMENT a11 ON a11.sps_it == a11.sps_id and a11.measurement_set_name == a4.measurement_set_name
WHERE
              a0.lot7 = 'L210060' 
 AND      (SELECT lrc99.last_pass FROM F_LOT_RUN_CARD lrc99 where lrc99.lot =a0.lot AND lrc99.operation = a0.operation AND lrc99.site_prevout_date=a0.prev_moveout_time and rownum<=1) = 'Y' 
 AND      a0.rework_latest_flag = 'Y' 
 AND      a3.valid_flag <> 'I' 
 AND      a5.latest_flag = 'Y' 
 AND      a3.latest_flag = 'Y' 
 AND      a5.value is not NULL"""

In [ ]:
sql2 = """
SELECT 
          a0.lot7 AS lot7
         ,a0.lot_type AS lot_type
         ,a0.lot_virtual_line AS lot_virtual_line
         ,a0.operation AS spc_operation
         ,a0.devrevstep AS devrevstep
         ,To_Char(a0.data_collection_time,'yyyy-mm-dd hh24:mi:ss') AS lot_data_collect_date
         ,a1.entity AS entity
         ,To_Char(a1.data_collection_time,'yyyy-mm-dd hh24:mi:ss') AS entity_data_collect_date
         ,a1.entity_sequence AS entity_sequence
         ,a2.monitor_set_name AS monitor_set_name
         ,a3.measurement_set_name AS measurement_set_name
         ,a2.test_name AS test_name
         ,a2.spc_data_id AS spc_data_id
         ,a5.chart_type AS chart_type
         ,a5.spc_chart_category AS spc_chart_category
         ,a5.spc_chart_subset AS spc_chart_subset
         ,a5.value AS chart_value
FROM 
P_SPC_LOT a0
LEFT JOIN P_SPC_ENTITY a1 ON a1.spcs_id = a0.spcs_id AND a1.entity_sequence=1
INNER JOIN P_SPC_SESSION a2 ON a2.spcs_id = a0.spcs_id AND a2.data_collection_time = a0.data_collection_time
INNER JOIN P_SPC_MEASUREMENT_SET a3 ON a3.spcs_id = a2.spcs_id
LEFT JOIN P_SPC_CHART_POINT a5 ON a5.spcs_id = a3.spcs_id AND a5.measurement_set_name = a3.measurement_set_name
WHERE
              (SELECT lrc99.last_pass FROM F_LOT_RUN_CARD lrc99 where lrc99.lot =a0.lot AND lrc99.operation = a0.operation AND lrc99.site_prevout_date=a0.prev_moveout_time and rownum<=1) = 'Y' 
 AND      a3.latest_flag = 'Y' 
 AND      a5.latest_flag = 'Y' 
 AND      a0.data_collection_time >= (SYSDATE - 5/24) 
 AND      a0.lot Like 'L%' """

In [3]:
con = PyUber.connect("F32_PROD_XEUS")

In [4]:
df = pd.read_sql(sql, con)

C:\Anaconda3\envs\WI\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [5]:
df = df[df['MONITOR_TYPE'] == "WIP MONITOR"]


In [8]:
df['CHART_WAFER3'].value_counts()/df.shape[0]

279    0.082959
282    0.007100
280    0.002157
291    0.000539
302    0.000539
300    0.000539
290    0.000180
295    0.000180
287    0.000180
283    0.000090
284    0.000090
Name: CHART_WAFER3, dtype: float64

In [9]:
dfnl = df[df['CHART_WAFER3'].isna()]

In [10]:
dfnl = dfnl[dfnl['CHART_PT_STATUS'] == 'V']

In [11]:
dfnl['colnames'] = dfnl['SPC_OPERATION'] + '`' + dfnl['TEST_NAME'] + '`' + dfnl['MONITOR_SET_NAME'] + '`' + dfnl['MEASUREMENT_SET_NAME'] + '`' + dfnl['SPC_CHART_CATEGORY'] + '`' + dfnl['SPC_CHART_SUBSET'] + '`' + dfnl['CHART_TYPE'] + '`' + dfnl['PARAMETER_CLASS'] + '`' + dfnl['CHART_ON']
#dfnl.pivot(index=['LOT7'], columns='colnames', values = 'CHART_VALUE')

In [12]:
sub = dfnl.groupby(['LOT7', 'colnames']).size()

In [13]:
r = sub[sub > 1].reset_index()
#sub[sub['count']>1].head()

In [14]:
dup = dfnl[dfnl['colnames'].isin(r['colnames'].unique())]

In [19]:
dup['CHART_VALUE']

1247     58.700000
1248     33.100000
1249     58.800000
1250     33.050000
1251     58.700000
1252     33.100000
1253     58.800000
1254     33.050000
5827    104.242857
5828    105.200000
5829    104.800000
5830    105.812500
6018      0.000000
6019      0.000000
6020      0.000000
6021      0.000000
6284      1.000000
6285      1.000000
6361      1.000000
6362      1.000000
Name: CHART_VALUE, dtype: float64

In [57]:
dup['colnames'].unique().size, dfnl['colnames'].unique().size

(10, 9760)

In [212]:
m = dup.iloc[1] != dup.iloc[0]
print(m)

LOT7                    False
DEVREVSTEP              False
LAST_PASS               False
REWORK_LATEST_FLAG      False
REWORK_FLAG             False
ENTITY                   True
CEID                    False
MONITOR_SET_NAME        False
DATA_COLLECT_DATE        True
AREA                    False
MONITOR_PROCESS         False
MON_SET_STATUS          False
SPC_DATA_ID              True
VIOLATION_FLAG          False
MON_SET_LATEST_FLAG     False
CHART_POINT_SEQ          True
CHART_VALUE             False
PROCESS_CHAMBER          True
CHART_WAFER              True
CHART_PT_LATEST_FLAG    False
CHART_PT_STATUS         False
CHART_TYPE              False
SPC_CHART_CATEGORY      False
SPC_CHART_SUBSET        False
LO_CONTROL_LMT          False
TARGET                   True
UP_CONTROL_LMT          False
TEST_NAME               False
SPC_OPERATION           False
CHART_WAFER3             True
MONITOR_TYPE            False
PARAMETER_CLASS         False
CHART_PARAMETER         False
CHART_ON  

In [213]:
dup.iloc[1][m], dup.iloc[0][m]

(ENTITY                                       DEA712_B
 DATA_COLLECT_DATE                 2022-03-09 14:25:03
 SPC_DATA_ID          6d17a895ec6f41e485d599a4ca96395a
 CHART_POINT_SEQ                                9701.0
 PROCESS_CHAMBER                                  None
 CHART_WAFER                                      None
 TARGET                                            NaN
 CHART_WAFER3                                     None
 Name: 1251, dtype: object,
 ENTITY                                       DEA712_A
 DATA_COLLECT_DATE                 2022-03-09 14:25:06
 SPC_DATA_ID          03b8f9291e0f49aa91fee91502e07d52
 CHART_POINT_SEQ                                9702.0
 PROCESS_CHAMBER                                  None
 CHART_WAFER                                      None
 TARGET                                            NaN
 CHART_WAFER3                                     None
 Name: 1247, dtype: object)

In [21]:
df = pd.read_sql("select * from P_SPC_CHARTPOINT_MEASUREMENT LIMIT 10", con)

In [25]:
df.dtypes

DATA_COLLECTION_WW               int64
SPCS_ID                        float64
DATA_COLLECTION_TIME    datetime64[ns]
CHART_ID                         int64
CHART_POINT_SEQ                float64
MONITOR_SET_NAME                object
MEASUREMENT_SET_NAME            object
SPC_MEASUREMENT_ID               int64
FACILITY                        object
SPC_CHART_SUBSET                object
SPC_CHART_CATEGORY              object
MEASUREMENT_ID                   int64
WAFER                           object
PRIMARY_ENTITY                  object
VALUE                          float64
WAFER_COORDINATE_X              object
WAFER_COORDINATE_Y              object
STATUS                          object
SECURITY_CODE                  float64
LOAD_DATE               datetime64[ns]
CS_CHECKSUM                     object
dtype: object